In [7]:
# Importar librerías necesarias
from google.colab import files
import zipfile
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import xgboost as xgb
import os
import time

# Forzar instalación de XGBoost 1.7.6
!pip install xgboost==1.7.6 --force-reinstall --quiet
print("XGBoost 1.7.6 instalado")

# Configurar Kaggle API y descargar datos
print("Por favor, sube el archivo kaggle.json")
uploaded = files.upload()

if 'kaggle.json' in uploaded:
    !mkdir -p ~/.kaggle
    !mv kaggle.json ~/.kaggle/
    !chmod 600 ~/.kaggle/kaggle.json
    print("kaggle.json configurado correctamente")
else:
    raise FileNotFoundError("No se subió kaggle.json. Descarga desde Kaggle: Account > API > Create New API Token")

!pip install kaggle --quiet
!kaggle competitions download -c udea-ai-4-eng-20251-pruebas-saber-pro-colombia
if os.path.exists("udea-ai-4-eng-20251-pruebas-saber-pro-colombia.zip"):
    with zipfile.ZipFile("udea-ai-4-eng-20251-pruebas-saber-pro-colombia.zip", "r") as zip_ref:
        zip_ref.extractall("datos")
    print("Datos extraídos en datos/")
    if os.path.exists('datos/train.csv'):
        print(f"train.csv encontrado, tamaño: {os.path.getsize('datos/train.csv')} bytes")
    if os.path.exists('datos/test.csv'):
        print(f"test.csv encontrado, tamaño: {os.path.getsize('datos/test.csv')} bytes")
    else:
        raise FileNotFoundError("test.csv no se encuentra en datos/")
else:
    raise FileNotFoundError("Fallo al descargar el ZIP. Verifica tus credenciales de Kaggle")

# Cargar datos
start_time = time.time()
try:
    train = pd.read_csv('datos/train.csv', engine='python', encoding='utf-8')
    test = pd.read_csv('datos/test.csv', engine='python', encoding='utf-8')
    print("Archivos cargados correctamente")
    test_ids = test['ID']
except Exception as e:
    print(f"Error al cargar archivos: {e}")
    raise

# Eliminar columna duplicada si existe
if 'FAMI_TIENEINTERNET.1' in train.columns:
    print("Eliminando columna duplicada FAMI_TIENEINTERNET.1 en train")
    train = train.drop('FAMI_TIENEINTERNET.1', axis=1)
if 'FAMI_TIENEINTERNET.1' in test.columns:
    print("Eliminando columna duplicada FAMI_TIENEINTERNET.1 en test")
    test = test.drop('FAMI_TIENEINTERNET.1', axis=1)

# Estandarizar valores
train['FAMI_ESTRATOVIVIENDA'] = train['FAMI_ESTRATOVIVIENDA'].str.lower().fillna('sin estrato')
test['FAMI_ESTRATOVIVIENDA'] = test['FAMI_ESTRATOVIVIENDA'].str.lower().fillna('sin estrato')

# Codificación de frecuencia para ESTU_PRGM_ACADEMICO y ESTU_PRGM_DEPARTAMENTO
program_counts = train['ESTU_PRGM_ACADEMICO'].value_counts()
train['ESTU_PRGM_ACADEMICO_FREQ'] = train['ESTU_PRGM_ACADEMICO'].map(program_counts)
test['ESTU_PRGM_ACADEMICO_FREQ'] = test['ESTU_PRGM_ACADEMICO'].map(program_counts).fillna(program_counts.mean())

dept_counts = train['ESTU_PRGM_DEPARTAMENTO'].value_counts()
train['ESTU_PRGM_DEPARTAMENTO_FREQ'] = train['ESTU_PRGM_DEPARTAMENTO'].map(dept_counts)
test['ESTU_PRGM_DEPARTAMENTO_FREQ'] = test['ESTU_PRGM_DEPARTAMENTO'].map(dept_counts).fillna(dept_counts.mean())

# Agrupar programas raros
rare_programs = program_counts[program_counts < 25].index
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'otro' if x in rare_programs else x)
test['ESTU_PRGM_ACADEMICO'] = test['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'otro' if x in rare_programs else x)

# Manejar valores faltantes
for df in [train, test]:
    df['ESTU_VALORMATRICULAUNIVERSIDAD'] = df['ESTU_VALORMATRICULAUNIVERSIDAD'].fillna('no pagó matrícula')
    df['ESTU_HORASSEMANATRABAJA'] = df['ESTU_HORASSEMANATRABAJA'].fillna('0')
    df['FAMI_EDUCACIONPADRE'] = df['FAMI_EDUCACIONPADRE'].fillna('No sabe')
    df['FAMI_EDUCACIONMADRE'] = df['FAMI_EDUCACIONMADRE'].fillna('No sabe')
    df['FAMI_TIENEINTERNET'] = df['FAMI_TIENEINTERNET'].fillna('No')
    df['FAMI_TIENECOMPUTADOR'] = df['FAMI_TIENECOMPUTADOR'].fillna('No')
    df['FAMI_TIENELAVADORA'] = df['FAMI_TIENELAVADORA'].fillna('No')
    df['FAMI_TIENEAUTOMOVIL'] = df['FAMI_TIENEAUTOMOVIL'].fillna('No')
    df['ESTU_PAGOMATRICULAPROPIO'] = df['ESTU_PAGOMATRICULAPROPIO'].fillna('No')
    df['ESTU_PRIVADO_LIBERTAD'] = df['ESTU_PRIVADO_LIBERTAD'].fillna('N')

# Codificación ordinal
valormatricula_map = {
    'no pagó matrícula': 0,
    'menos de 500 mil': 1,
    'entre 500 mil y menos de 1 millón': 2,
    'entre 1 millón y menos de 2.5 millones': 3,
    'entre 2.5 millones y menos de 4 millones': 4,
    'entre 4 millones y menos de 5.5 millones': 5,
    'entre 5.5 millones y menos de 7 millones': 6,
    'más de 7 millones': 7
}
train['ESTU_VALORMATRICULAUNIVERSIDAD'] = train['ESTU_VALORMATRICULAUNIVERSIDAD'].str.lower().map(valormatricula_map)
test['ESTU_VALORMATRICULAUNIVERSIDAD'] = test['ESTU_VALORMATRICULAUNIVERSIDAD'].str.lower().map(valormatricula_map)

horastrabajo_map = {
    '0': 0,
    'menos de 10 horas': 1,
    'entre 11 y 20 horas': 2,
    'entre 21 y 30 horas': 3,
    'más de 30 horas': 4
}
train['ESTU_HORASSEMANATRABAJA'] = train['ESTU_HORASSEMANATRABAJA'].str.lower().map(horastrabajo_map)
test['ESTU_HORASSEMANATRABAJA'] = test['ESTU_HORASSEMANATRABAJA'].str.lower().map(horastrabajo_map)

# Aplicar Label Encoding
label_cols = ['ESTU_PRGM_ACADEMICO', 'ESTU_PRGM_DEPARTAMENTO', 'FAMI_EDUCACIONPADRE', 'FAMI_EDUCACIONMADRE']
le_dict = {}
for col in label_cols:
    le = LabelEncoder()
    le.fit(train[col])
    train[col] = le.transform(train[col])
    test[col] = test[col].map(lambda s: '<unknown>' if s not in le.classes_ else s)
    le.classes_ = np.append(le.classes_, '<unknown>')
    test[col] = le.transform(test[col])
    le_dict[col] = le

# Aplicar One-Hot Encoding
one_hot_cols = ['FAMI_ESTRATOVIVIENDA', 'FAMI_TIENEINTERNET', 'FAMI_TIENECOMPUTADOR',
                'FAMI_TIENELAVADORA', 'FAMI_TIENEAUTOMOVIL', 'ESTU_PAGOMATRICULAPROPIO',
                'ESTU_PRIVADO_LIBERTAD']
train = pd.get_dummies(train, columns=one_hot_cols, drop_first=True)
test = pd.get_dummies(test, columns=one_hot_cols, drop_first=True)

# Asegurar que test tenga las mismas columnas que train
missing_cols = set(train.columns) - set(test.columns) - {'ID', 'RENDIMIENTO_GLOBAL'}
for col in missing_cols:
    test[col] = 0
test_preprocessed = test[train.drop(['ID', 'RENDIMIENTO_GLOBAL'], axis=1).columns]

# Crear interacciones selectivas
num_cols = ['coef_1', 'coef_2', 'coef_3', 'coef_4']
for col1, col2 in [('coef_1', 'coef_2'), ('coef_3', 'coef_4'), ('ESTU_VALORMATRICULAUNIVERSIDAD', 'ESTU_HORASSEMANATRABAJA')]:
    col_name = f"{col1}_x_{col2}"
    train[col_name] = train[col1] * train[col2]
    test_preprocessed[col_name] = test_preprocessed[col1] * test_preprocessed[col2]

# Normalizar columnas numéricas con RobustScaler
all_num_cols = num_cols + [col for col in train.columns if col.startswith('coef_') and '_x_' in col] + ['ESTU_PRGM_ACADEMICO_FREQ', 'ESTU_PRGM_DEPARTAMENTO_FREQ', 'ESTU_VALORMATRICULAUNIVERSIDAD_x_ESTU_HORASSEMANATRABAJA']
scaler = RobustScaler()
train[all_num_cols] = scaler.fit_transform(train[all_num_cols])
test_preprocessed[all_num_cols] = scaler.transform(test_preprocessed[all_num_cols])

# Eliminar columnas duplicadas
train = train.loc[:, ~train.columns.duplicated()]
test_preprocessed = test_preprocessed.loc[:, ~test_preprocessed.columns.duplicated()]

# Verificar datos
print('Columnas en train:', train.columns.tolist())
print('Tipos de datos en train:\n', train.dtypes)
print('Columnas en test_preprocessed:', test_preprocessed.columns.tolist())
print('Tipos de datos en test_preprocessed:\n', test_preprocessed.dtypes)
for col in train.columns:
    if train[col].dtype == 'object' and col not in ['ID', 'RENDIMIENTO_GLOBAL']:
        print(f"Columna con strings en train: {col}, valores: {train[col].unique()}")
for col in test_preprocessed.columns:
    if test_preprocessed[col].dtype == 'object':
        print(f"Columna con strings en test_preprocessed: {col}, valores: {test_preprocessed[col].unique()}")
print(f"Tiempo de carga y preprocesamiento: {(time.time() - start_time) / 60:.2f} minutos")

# Separar características y variable objetivo
X_99 = train.drop(['ID', 'RENDIMIENTO_GLOBAL'], axis=1)
y_99 = train['RENDIMIENTO_GLOBAL']

# Convertir etiquetas categóricas a numéricas
label_encoder_y = LabelEncoder()
y_99 = label_encoder_y.fit_transform(y_99)
print(f"Clases de LabelEncoder: {label_encoder_y.classes_}")

# Calcular scale_pos_weight para la clase minoritaria
class_counts = pd.Series(y_99).value_counts()
scale_pos_weight = class_counts.max() / class_counts.min() if len(class_counts) > 1 else 1
print(f"scale_pos_weight: {scale_pos_weight:.4f}")

# Dividir en conjunto de entrenamiento y validación
X_train_99, X_val_99, y_train_99, y_val_99 = train_test_split(X_99, y_99, test_size=0.2, random_state=42, stratify=y_99)

print('Tamaño del conjunto de entrenamiento (99):', X_train_99.shape)
print('Tamaño del conjunto de validación (99):', X_val_99.shape)

# Entrenar modelo XGBoost
start_time = time.time()
xgb_model = xgb.XGBClassifier(
    n_estimators=800,
    max_depth=10,
    learning_rate=0.05,
    scale_pos_weight=scale_pos_weight,
    subsample=0.8,
    colsample_bytree=0.8,
    eval_metric='mlogloss',
    random_state=42,
    n_jobs=-1
)
xgb_model.fit(X_train_99, y_train_99)
print(f"Tiempo de entrenamiento de XGBoost: {(time.time() - start_time) / 60:.2f} minutos")

# Evaluar en conjunto de validación
y_pred_99 = xgb_model.predict(X_val_99)
print('Precisión en validación (99):', accuracy_score(y_val_99, y_pred_99))
print('Reporte de clasificación (99):\n', classification_report(y_val_99, y_pred_99, target_names=label_encoder_y.classes_))

# Generar predicciones para test.csv
test_pred_99 = xgb_model.predict(test_preprocessed)

# Verificar predicciones
print(f"Primeras 10 predicciones (numéricas): {test_pred_99[:10]}")

# Convertir predicciones numéricas a etiquetas categóricas
test_pred_99 = label_encoder_y.inverse_transform(test_pred_99)

# Verificar etiquetas predichas
print(f"Primeras 10 etiquetas predichas: {test_pred_99[:10]}")

# Crear archivo de sumisión
submission_99 = pd.DataFrame({'ID': test_ids, 'RENDIMIENTO_GLOBAL': test_pred_99})
submission_99.to_csv('submission.csv', index=False)
print('Archivo de sumisión guardado como submission.csv')

# Verificar que el archivo se guardó
if os.path.exists('submission.csv'):
    print(f"submission.csv existe, tamaño: {os.path.getsize('submission.csv')} bytes")
else:
    print("Error: submission.csv no se guardó correctamente")

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cupy-cuda12x 13.3.0 requires numpy<2.3,>=1.22, but you have numpy 2.3.1 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.3.1 which is incompatible.
plotnine 0.14.6 requires scipy<1.16.0,>=1.8.0, but you have scipy 1.16.0 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 2.3.1 which is incompatible.
XGBoost 1.7.6 instalado
Por favor, sube el archivo kaggle.json


Saving kaggle.json to kaggle.json
kaggle.json configurado correctamente
  0% 0.00/29.9M [00:00<?, ?B/s]
100% 29.9M/29.9M [00:00<00:00, 783MB/s]
Datos extraídos en datos/
train.csv encontrado, tamaño: 143732449 bytes
test.csv encontrado, tamaño: 59185250 bytes
Archivos cargados correctamente
Eliminando columna duplicada FAMI_TIENEINTERNET.1 en train
Eliminando columna duplicada FAMI_TIENEINTERNET.1 en test
Columnas en train: ['ID', 'PERIODO', 'ESTU_PRGM_ACADEMICO', 'ESTU_PRGM_DEPARTAMENTO', 'ESTU_VALORMATRICULAUNIVERSIDAD', 'ESTU_HORASSEMANATRABAJA', 'FAMI_EDUCACIONPADRE', 'FAMI_EDUCACIONMADRE', 'RENDIMIENTO_GLOBAL', 'coef_1', 'coef_2', 'coef_3', 'coef_4', 'ESTU_PRGM_ACADEMICO_FREQ', 'ESTU_PRGM_DEPARTAMENTO_FREQ', 'FAMI_ESTRATOVIVIENDA_estrato 2', 'FAMI_ESTRATOVIVIENDA_estrato 3', 'FAMI_ESTRATOVIVIENDA_estrato 4', 'FAMI_ESTRATOVIVIENDA_estrato 5', 'FAMI_ESTRATOVIVIENDA_estrato 6', 'FAMI_ESTRATOVIVIENDA_sin estrato', 'FAMI_TIENEINTERNET_Si', 'FAMI_TIENECOMPUTADOR_Si', 'FAMI_TIENELAVADORA